In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


In [5]:

import csv
from pprint import pprint
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


# 관심 분야를 코드에 직접 명시
subject = "운동"  # 원하는 분야를 여기서 바꾸세요 (예: 음식, 스포츠, 영화 등)

# 콤마 구분 리스트 출력 파서 초기화
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

# 프롬프트 템플릿 설정 (한글 및 한국 관련 지시 포함)
prompt = PromptTemplate(
    template="사용자가 관심 있어 하는 분야는 '{subject}'입니다. 이와 관련된 한국의 유명한 장소나 활동 5가지를 추천해주세요.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

pprint(prompt.partial_variables)

# Groq API 모델 설정
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

# 체인 생성
chain = prompt | model | output_parser

# 체인 실행
result = chain.invoke({"subject": subject})

# 결과 출력
print(f"\n'{subject}' 관련 한국의 유명한 장소 또는 활동 추천:")
for item in result:
    print(f"- {item}")

# 결과를 CSV로 저장
csv_filename = f"./data/korea_{subject}_recommendations.csv"
os.makedirs(os.path.dirname(csv_filename), exist_ok=True)

with open(csv_filename, "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["추천 항목"])
    for item in result:
        writer.writerow([item])

print(f"\n결과가 '{csv_filename}' 파일로 저장되었습니다.")


{'format_instructions': 'Your response should be a list of comma separated '
                        'values, eg: `foo, bar, baz` or `foo,bar,baz`'}

'운동' 관련 한국의 유명한 장소 또는 활동 추천:
- Here are 5 famous Korean places or activities related to exercise: 
- 한라산 등산
- 부산 해운대 해수욕장
- 경주 보문단지 마라톤
- 서울 올림픽공원
- 파주 MTB파크

결과가 './data/korea_운동_recommendations.csv' 파일로 저장되었습니다.
